# Cenário 1

In [1]:
import threading

# Define uma classe para representar uma urna eletrônica
class UrnaEletronica:
    def __init__(self, ident):
        self.ident = ident
        self.votos_a = 0
        self.votos_b = 0

    def adicionar_voto(self, voto):
        if voto == "A":
            self.votos_a += 1
        else:
            self.votos_b += 1

    def __str__(self):
        return f"Urna {self.ident}: Votos A={self.votos_a}, Votos B={self.votos_b}"

# Função para gerar números aleatórios entre 0 e 1
def random_generator():
    seed = 1
    while True:
        seed = (1103515245 * seed + 12345) % (2**31)
        yield seed / (2**31)

# Função que processa os votos em paralelo para cada urna
def processar_votos_paralelo(urna, num_votos, random_gen):
    for _ in range(num_votos):
        random_num = next(random_gen)
        if random_num < 0.5:
            urna.adicionar_voto("A")
        else:
            urna.adicionar_voto("B")

def main():
    # Define o número de urnas eletrônicas e de votos em cada urna
    num_urnas = 10
    num_votos = 100

    # Inicializa as variáveis de contagem de votos para cada candidato
    total_votos_a = 0
    total_votos_b = 0

    # Cria uma lista para armazenar as leituras de cada urna monitorada
    leituras = []

    # Inicializa o gerador de números aleatórios
    random_gen = random_generator()

    # Cria uma lista para armazenar as threads
    threads = []

    # Loop através das urnas eletrônicas
    for i in range(num_urnas):
        urna = UrnaEletronica(i+1)

        # Cria a thread para processar os votos da urna
        thread = threading.Thread(target=processar_votos_paralelo, args=(urna, num_votos, random_gen))
        threads.append(thread)

        # Inicia a thread
        thread.start()

        # Adiciona os votos da urna ao total de votos para cada candidato
        total_votos_a += urna.votos_a
        total_votos_b += urna.votos_b

        # Adiciona as leituras da urna à lista de leituras
        leituras.append((urna.ident, urna.votos_a, urna.votos_b))

    # Aguarda a conclusão de todas as threads
    for thread in threads:
        thread.join()

    # Ordena a lista de leituras por votos do candidato A em ordem crescente
    leituras_ord = sorted(leituras, key=lambda x: x[1])

    # Imprime a lista ordenada de leituras
    print("\nUrna\tVotos A\tVotos B")
    for urna in leituras_ord:
        print(f"{urna[0]}\t{urna[1]}\t{urna[2]}")

    # Imprime o total de votos para cada candidato em todas as urnas
    print(f"\nTotal de votos: Candidato A recebeu {total_votos_a} votos; Candidato B recebeu {total_votos_b} votos.")

    # Executa uma funcionalidade extra que tem complexidade O(N!)
    func_extra(num_urnas)

def func_extra(num_urnas):
    # Gera todas as permutações possíveis das urnas
    urnas = list(range(1, num_urnas+1))
    perms = permute(urnas)

    # Imprime o número total de permutações geradas
    print(f"\nNúmero total de permutações: {len(perms)}")

def permute(lst):
    if len(lst) == 0:
        return []
    if len(lst) == 1:
        return [lst]
    l = []
    for i in range(len(lst)):
        m = lst[i]
        rem_lst = lst[:i] + lst[i+1:]
        for p in permute(rem_lst):
            l.append([m] + p)
    return l

if __name__ == "__main__":
    main()



Urna	Votos A	Votos B
8	39	61
6	44	56
7	45	55
9	46	54
10	49	51
4	50	50
5	50	50
1	51	49
2	52	48
3	54	46

Total de votos: Candidato A recebeu 480 votos; Candidato B recebeu 520 votos.

Número total de permutações: 3628800


# Cenário 2

In [2]:
import threading

# Define uma classe para representar uma urna eletrônica
class UrnaEletronica:
    def __init__(self, ident):
        self.ident = ident
        self.votos_a = 0
        self.votos_b = 0

    def adicionar_voto(self, voto):
        if voto == "A":
            self.votos_a += 1
        else:
            self.votos_b += 1

    def __str__(self):
        return f"Urna {self.ident}: Votos A={self.votos_a}, Votos B={self.votos_b}"

# Define uma classe para representar um servidor de borda
class ServidorBorda:
    def __init__(self, urnas):
        self.urnas = urnas

    def processar_votos(self, num_votos, random_gen):
        # Processa os votos em paralelo para cada urna
        for urna in self.urnas:
            for _ in range(num_votos):
                random_num = next(random_gen)
                if random_num < 0.5:
                    urna.adicionar_voto("A")
                else:
                    urna.adicionar_voto("B")

    def calcular_total_votos(self):
        # Calcula o total de votos para cada candidato em todas as urnas
        total_votos_a = sum(urna.votos_a for urna in self.urnas)
        total_votos_b = sum(urna.votos_b for urna in self.urnas)
        return total_votos_a, total_votos_b

    def func_extra(self, num_urnas):
        # Executa uma funcionalidade extra que tem complexidade O(N!)

        # Gera todas as permutações possíveis das urnas
        urnas = list(range(1, num_urnas + 1))
        perms = self.permute(urnas)

        # Retorna o número total de permutações geradas
        return len(perms)

    def permute(self, lst):
        # Função auxiliar para gerar todas as permutações de uma lista
        if len(lst) == 0:
            return []
        if len(lst) == 1:
            return [lst]
        l = []
        for i in range(len(lst)):
            m = lst[i]
            rem_lst = lst[:i] + lst[i + 1:]
            for p in self.permute(rem_lst):
                l.append([m] + p)
        return l

    def executar_calculo(self, num_votos, random_gen):
        self.processar_votos(num_votos, random_gen)

# Define uma classe para representar um servidor na nuvem
class ServidorNuvem:
    def __init__(self, servidores_borda):
        self.servidores_borda = servidores_borda
        self.resultado_da_complexidade = None

    def receber_resultados(self):
        # Recebe os resultados dos cálculos das bordas e exibe os resultados
        leituras = []
        for servidor_borda in self.servidores_borda:
            leituras.extend(
                (urna.ident, urna.votos_a, urna.votos_b) for urna in servidor_borda.urnas
            )

        # Ordena a lista de leituras por votos do candidato A em ordem crescente
        leituras_ord = sorted(leituras, key=lambda x: x[1])

        # Imprime a lista ordenada de leituras
        print("\nUrna\tVotos A\tVotos B")
        for urna in leituras_ord:
            print(f"{urna[0]}\t{urna[1]}\t{urna[2]}")

        total_votos_a, total_votos_b = self.calcular_total_votos()
        print(
            f"\nTotal de votos: Candidato A recebeu {total_votos_a} votos; Candidato B recebeu {total_votos_b} votos."
        )

    def calcular_total_votos(self):
        # Calcula o total de votos para cada candidato em todas as urnas
        total_votos_a = sum(
            servidor_borda.calcular_total_votos()[0] for servidor_borda in self.servidores_borda
        )
        total_votos_b = sum(
            servidor_borda.calcular_total_votos()[1] for servidor_borda in self.servidores_borda
        )
        return total_votos_a, total_votos_b

    def armazenar_resultado_da_complexidade(self, resultado):
        # Método para armazenar o resultado da complexidade na variável resultado_da_complexidade
        self.resultado_da_complexidade = resultado

    def imprimir_resultado_da_complexidade(self):
        # Método para imprimir o resultado da complexidade armazenado na variável resultado_da_complexidade
        if self.resultado_da_complexidade is not None:
            # Verifica se o resultado da complexidade possui um valor válido
            print(f"Resultado da Complexidade: {self.resultado_da_complexidade}")
        else:
            # Caso o resultado da complexidade não possua valor, exibe uma mensagem informando isso
            print("Resultado da Complexidade não possui valor.")


# Função para gerar números aleatórios entre 0 e 1
def random_generator():
    seed = 1
    while True:
        seed = (1103515245 * seed + 12345) % (2 ** 31)
        yield seed / (2 ** 31)

def main():
    # Define o número de urnas eletrônicas e de votos em cada urna
    num_urnas = 10
    num_votos = 100

    # Divide as urnas entre os servidores de borda
    urnas = [UrnaEletronica(i + 1) for i in range(num_urnas)]
    servidor_borda1 = ServidorBorda(urnas[:5])
    servidor_borda2 = ServidorBorda(urnas[5:])

    # Cria o servidor na nuvem
    servidor_nuvem = ServidorNuvem([servidor_borda1, servidor_borda2])

    # Inicializa o gerador de números aleatórios
    random_gen = random_generator()

    # Executa o cálculo nas bordas e envia o resultado para a nuvem
    servidor_borda1.executar_calculo(num_votos, random_gen)
    servidor_borda2.executar_calculo(num_votos, random_gen)

    # Exibe os resultados recebidos na nuvem
    servidor_nuvem.receber_resultados()

    # Executa a funcionalidade extra nas bordas
    resultado_da_complexidade = servidor_borda1.func_extra(num_urnas)
    servidor_nuvem.armazenar_resultado_da_complexidade(resultado_da_complexidade)

    # Imprime o resultado da complexidade na classe ServidorNuvem
    servidor_nuvem.imprimir_resultado_da_complexidade()

if __name__ == "__main__":
    main()



Urna	Votos A	Votos B
8	39	61
6	44	56
7	45	55
9	46	54
10	49	51
4	50	50
5	50	50
1	51	49
2	52	48
3	54	46

Total de votos: Candidato A recebeu 480 votos; Candidato B recebeu 520 votos.
Resultado da Complexidade: 3628800


### CENÁRIO 1 – qual é o impacto para a complexidade da solução de sensoriamento quando o processamento dos dados se concentra totalmente no programa principal?

A complexidade da solução de sensoriamento é relativamente menor. O programa principal é responsável por realizar todas as operações de processamento dos dados dos sensores. Isso resulta em um código mais simples, pois não há necessidade de coordenar várias threads ou pontos de processamento paralelo. A complexidade se concentra principalmente na lógica do programa principal, lidando com a leitura dos dados dos sensores, o processamento e o armazenamento dos resultados.

### CENÁRIO 2 – qual é o impacto para a complexidade da solução de sensoriamento quando o processamento é distribuído do programa principal para cada ponto ou Thread de processamento paralelo?

A complexidade da solução de sensoriamento pode aumentar. Nesse cenário, é necessário dividir o processamento dos dados em várias threads separadas, cada uma responsável por processar uma parte dos dados dos sensores. Isso requer o gerenciamento adequado das threads. Além de distribuirmos o processamento com uso de edge computing, o que torna viável o trabalho conjunto entre borda e servidor, descongestionando o uso de um só meio e não sobrecarregando a nuvem, o que muito usado em IoT.

## Vantages e desvantagens

CENÁRIO 1:

Complexidade das funções/métodos: No CENÁRIO 1, as funções/métodos têm uma complexidade geralmente menor, pois o processamento dos dados ocorre diretamente no programa principal. O código é relativamente mais simples e direto, sem a necessidade de dividir o processamento em threads separadas.

Vantagens do CENÁRIO 1:

Implementação mais simples: O programa principal lida com todo o processamento, resultando em um código mais conciso e de fácil compreensão.
Menos gerenciamento de threads: Não há necessidade de gerenciar várias threads separadas para o processamento paralelo.
Desvantagens do CENÁRIO 1:

Escalabilidade limitada: O processamento ocorre sequencialmente no programa principal, o que pode levar a um desempenho inferior quando a carga de dados aumenta.
Potencial sobrecarga do programa principal: Se o processamento dos dados for complexo ou demorado, o programa principal pode ser sobrecarregado, afetando a capacidade de resposta do sistema.
CENÁRIO 2 (código fornecido anteriormente):

Complexidade das funções/métodos: No CENÁRIO 2, a complexidade das funções/métodos pode ser maior, pois o processamento é dividido em threads separadas, exigindo a coordenação e sincronização adequadas entre elas.

Vantagens do CENÁRIO 2:

Melhor desempenho: O processamento paralelo distribuído em várias threads pode melhorar o desempenho geral, permitindo que várias urnas sejam processadas simultaneamente.
Maior escalabilidade: Com o processamento distribuído, o sistema pode lidar com cargas de dados maiores de forma mais eficiente.
Desvantagens do CENÁRIO 2:

Complexidade de gerenciamento de threads: A implementação de threads requer sincronização adequada, gerenciamento de concorrência e resolução de possíveis problemas, como condições de corrida e deadlocks.
Maior complexidade de código: A divisão do processamento em várias threads e a necessidade de coordenação entre elas podem aumentar a complexidade do código e a possibilidade de erros.